## 1 概念


2022年，stable diffusion发布，开启了AIGC的时代，各种视觉生成模型/应用层出不穷，比较典型的应用还有Midjourney，以及DALL-E2，现在openai已经逐步开始测试DALL-E3。基于diffusion的图像生成，视频生成逐步成为目前的一个研究和应用趋势。  
以Chatgpt为典型的这类模型是文本生成模型，那么视觉生成模型就不难理解，顾名思义，针对视觉任务（图像、视频），进行内容生成，根据输入的文字、图像生成图像、视频等。 

![](https://github.com/CompVis/stable-diffusion/raw/main/assets/stable-samples/txt2img/merged-0006.png)
<center>stable diffusion</center>

[]()  midjourney

### 1.1 什么是视觉生成大模型？

视觉生成是计算机视觉领域的一个重要研究方向，指根据特定的输入（随机噪声、文本、图像和视频等）生成与目标分布相匹配的图像或视频，可以实现对图像和视频的生成、美化、渲染和重建等操作。  
视觉生成技术在日常生活中有广泛的应用，比如视觉设计、图像/视频制作等。  
视觉生成已经成为机器学习领域热门的方向之一。  
视觉生成的目标是生成尽可能真实的数据，其关键在于构造生成模型。


[深度对抗视觉生成综述](http://www.cjig.cn/jig/ch/reader/create_pdf.aspx?file_no=20211201&flag=1&year_id=2021&quarter_id=12)

### 1.2 视觉生成模型与自然语言生成模型的区别？

[自然语言生成综述](http://www.joca.cn/CN/10.11772/j.issn.1001-9081.2020071069)

自然语言最终要生成的是文本信息。  
典型任务有：文本到文本生成、数据到文本生成和图像到文本生成。  
应用场景  
NLG 技术具有极为广泛的应用价值，应 用于智能问答对话系统和机器翻译系统时，可实现更为智能 便捷的人机交互；应用于机器新闻写作［2］ 、医学诊断报告生 成［3］ 和天气预报生成［4］ 等领域时，可实现文章报告自动撰写， 有效减轻人工工作；应用于文章摘要、文本复述领域时，可为 读者创造快速阅读条件等。


## 2 方法

### 2.1 生成对抗网络（GAN）

GAN网络，generative adversal network，全称是生成对抗网络。自2014年Ian Goodfellow提出了GAN以来，对GAN的各种研究如火如荼，GAN的各种变体也层出不穷。

![](https://pic1.zhimg.com/80/v2-8fb3a568c252eeae621bd250ccb8b2cc_1440w.webp)
<center>GAN相关论文发表情况</center>

GAN网络的核心在于“对抗”。GAN的设计思想就是一种对抗博弈的思想。

先举一个小例子说明一下这种对抗博弈的思想：两个人，一个人用剑进攻，一个人用盾防守，在对抗的过程中，进攻的一方不断地调整战术，试图攻破对方的防守；而另一个人，在对抗的过程中，不断地调整，或者把盾变得更加厚实，试图抵挡对方的攻击。  
在这个对抗的过程中，攻击者的技术会越来越高超。  
这就是对抗博弈的一个思想。

GAN网络的基本构成就是一个生成器和一个判别器，分别对应上述例子中的进攻者和防御者。  
以手写数字的数据集为例，定义一个模型作为生成器，一个模型（一般是分类器）作为判别器。生成器用来将输入的噪声信号转化为手写数字数据集中的图像；同时判别器判断图片是来自手写数据集还是生成器生成的（即图像的“真伪”）。  
在这种“对抗博弈”的过程中，生成器不断生成接近数据集中数据的图像，最终达到以假乱真的效果。

![GAN的基本结构](https://pic4.zhimg.com/80/v2-5ca6a701d92341b8357830cc176fb8a3_1440w.webp)
<center>GAN的基本结构</center>

以下是GAN网络的基本训练流程

In [ ]:
# 导包
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms

In [ ]:
# 数据准备
# 对数据做归一化 （-1， 1）
transform = transforms.Compose([
    transforms.ToTensor(),         # 将数据转换成Tensor格式，channel, high, witch,数据在（0， 1）范围内
    transforms.Normalize(0.5, 0.5) # 通过均值和方差将数据归一化到（-1， 1）之间
])

# 下载数据集
train_ds = torchvision.datasets.MNIST('data',
                                      train=True,
                                      transform=transform,
                                      download=True)
                                      
# 设置dataloader
dataloader = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)

# 返回一个批次的数据
imgs, _ = next(iter(dataloader))

# imgs的大小
imgs.shape

In [ ]:
# 定义生成器
# 输入是长度为 100 的 噪声（正态分布随机数）
# 输出为（1， 28， 28）的图片
# linear 1 :   100----256
# linear 2:    256----512
# linear 2:    512----28*28
# reshape:     28*28----(1, 28, 28)

class Generator(nn.Module): #创建的 Generator 类继承自 nn.Module
    def __init__(self): # 定义初始化方法
        super(Generator, self).__init__() #继承父类的属性
        self.main = nn.Sequential( #使用Sequential快速创建模型
                                  nn.Linear(100, 256),
                                  nn.ReLU(),
                                  nn.Linear(256, 512),
                                  nn.ReLU(),
                                  nn.Linear(512, 28*28),
                                  nn.Tanh()                     # 输出层使用Tanh()激活函数，使输出-1, 1之间
        )
    def forward(self, x):              # 定义前向传播 x 表示长度为100 的noise输入
        img = self.main(x)
        img = img.view(-1, 28, 28) #将img展平，转化成图片的形式，channel为1可写可不写
        return img


In [ ]:
# 定义判别器
## 输入为（1， 28， 28）的图片  输出为二分类的概率值，输出使用sigmoid激活 0-1
# BCEloss计算交叉熵损失

# nn.LeakyReLU   f(x) : x>0 输出 x， 如果x<0 ,输出 a*x  a表示一个很小的斜率，比如0.1
# 判别器中一般推荐使用 LeakyReLU

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
                                  nn.Linear(28*28, 512), #输入是28*28的张量，也就是图片
                                  nn.LeakyReLU(), # 小于0的时候保存一部分梯度
                                  nn.Linear(512, 256),
                                  nn.LeakyReLU(),
                                  nn.Linear(256, 1), # 二分类问题，输出到1上
                                  nn.Sigmoid()
        )
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.main(x)
        return x


In [ ]:
# 初始化模型、优化器及损失计算函数
# 定义设备
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 初始化模型
gen = Generator().to(device)
dis = Discriminator().to(device)
# 优化器
d_optim = torch.optim.Adam(dis.parameters(), lr=0.0001)
g_optim = torch.optim.Adam(gen.parameters(), lr=0.0001)
# 损失函数
loss_fn = torch.nn.BCELoss()

In [ ]:
# 绘图函数
def gen_img_plot(model, epoch, test_input):
    prediction = np.squeeze(model(test_input).detach().cpu().numpy())
    fig = plt.figure(figsize=(4, 4))
    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.imshow((prediction[i] + 1)/2) # 确保prediction[i] + 1)/2输出的结果是在0-1之间
        plt.axis('off')
    plt.show()
    
test_input = torch.randn(16, 100, device=device)


In [ ]:
# GAN的训练
# 保存每个epoch所产生的loss值
D_loss = []
G_loss = []

# 训练循环
for epoch in range(20): #训练20个epoch
   d_epoch_loss = 0 # 初始损失值为0
   g_epoch_loss = 0
   # len(dataloader)返回批次数，len(dataset)返回样本数
   count = len(dataloader)
   # 对dataloader进行迭代
   for step, (img, _) in enumerate(dataloader): # enumerate加序号
       img = img.to(device) #将数据上传到设备
       size = img.size(0) # 获取每一个批次的大小
       random_noise = torch.randn(size, 100, device=device)  # 随机噪声的大小是size个
       
       d_optim.zero_grad() # 将判别器前面的梯度归0
       
       real_output = dis(img)      # 判别器输入真实的图片，real_output是对真实图片的预测结果 
       
       # 得到判别器在真实图像上的损失
       # 判别器对于真实的图片希望输出的全1的数组，将真实的输出与全1的数组进行比较
       d_real_loss = loss_fn(real_output, 
                             torch.ones_like(real_output))      
       d_real_loss.backward() # 求解梯度
       
       
       gen_img = gen(random_noise)    
       # 判别器输入生成的图片，fake_output是对生成图片的预测
       # 优化的目标是判别器，对于生成器的参数是不需要做优化的，需要进行梯度阶段，detach()会截断梯度，
       # 得到一个没有梯度的Tensor，这一点很关键
       fake_output = dis(gen_img.detach()) 
       # 得到判别器在生成图像上的损失
       d_fake_loss = loss_fn(fake_output, 
                             torch.zeros_like(fake_output))      
       d_fake_loss.backward() # 求解梯度
       
       d_loss = d_real_loss + d_fake_loss # 判别器总的损失等于两个损失之和
       d_optim.step() # 进行优化
       
       g_optim.zero_grad() # 将生成器的所有梯度归0
       fake_output = dis(gen_img) # 将生成器的图片放到判别器中，此时不做截断，因为要优化生成器
       # 生层器希望生成的图片被判定为真
       g_loss = loss_fn(fake_output, 
                        torch.ones_like(fake_output))      # 生成器的损失
       g_loss.backward() # 计算梯度
       g_optim.step() # 优化
       
       # 将损失累加到定义的数组中，这个过程不需要计算梯度
       with torch.no_grad():
           d_epoch_loss += d_loss
           g_epoch_loss += g_loss
     
   # 计算每个epoch的平均loss，仍然使用这个上下文关联器
   with torch.no_grad():
       # 计算平均的loss值
       d_epoch_loss /= count
       g_epoch_loss /= count
       # 将平均loss放入到loss数组中
       D_loss.append(d_epoch_loss.item())
       G_loss.append(g_epoch_loss.item())
       # 打印当前的epoch
       print('Epoch:', epoch)
       # 调用绘图函数
       gen_img_plot(gen, epoch, test_input)


In [ ]:
# 整体代码

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms

# 对数据做归一化 （-1， 1）
transform = transforms.Compose([
    transforms.ToTensor(),         # 将数据转换成Tensor格式，channel, high, witch,数据在（0， 1）范围内
    transforms.Normalize(0.5, 0.5) # 通过均值和方差将数据归一化到（-1， 1）之间
])

# 下载数据集
train_ds = torchvision.datasets.MNIST('data',
                                      train=True,
                                      transform=transform,
                                      download=True)
                                      
# 设置dataloader
dataloader = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)

# 返回一个批次的数据
imgs, _ = next(iter(dataloader))

# imgs的大小
imgs.shape

# 输入是长度为 100 的 噪声（正态分布随机数）
# 输出为（1， 28， 28）的图片
# linear 1 :   100----256
# linear 2:    256----512
# linear 2:    512----28*28
# reshape:     28*28----(1, 28, 28)

class Generator(nn.Module): #创建的 Generator 类继承自 nn.Module
    def __init__(self): # 定义初始化方法
        super(Generator, self).__init__() #继承父类的属性
        self.main = nn.Sequential( #使用Sequential快速创建模型
                                  nn.Linear(100, 256),
                                  nn.ReLU(),
                                  nn.Linear(256, 512),
                                  nn.ReLU(),
                                  nn.Linear(512, 28*28),
                                  nn.Tanh()                     # 输出层使用Tanh()激活函数，使输出-1, 1之间
        )
    def forward(self, x):              # 定义前向传播 x 表示长度为100 的noise输入
        img = self.main(x)
        img = img.view(-1, 28, 28) #将img展平，转化成图片的形式，channel为1可写可不写
        return img

## 输入为（1， 28， 28）的图片  输出为二分类的概率值，输出使用sigmoid激活 0-1
# BCEloss计算交叉熵损失

# nn.LeakyReLU   f(x) : x>0 输出 x， 如果x<0 ,输出 a*x  a表示一个很小的斜率，比如0.1
# 判别器中一般推荐使用 LeakyReLU

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
                                  nn.Linear(28*28, 512), #输入是28*28的张量，也就是图片
                                  nn.LeakyReLU(), # 小于0的时候保存一部分梯度
                                  nn.Linear(512, 256),
                                  nn.LeakyReLU(),
                                  nn.Linear(256, 1), # 二分类问题，输出到1上
                                  nn.Sigmoid()
        )
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.main(x)
        return x

# 定义设备
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 初始化模型
gen = Generator().to(device)
dis = Discriminator().to(device)
# 优化器
d_optim = torch.optim.Adam(dis.parameters(), lr=0.0001)
g_optim = torch.optim.Adam(gen.parameters(), lr=0.0001)
# 损失函数
loss_fn = torch.nn.BCELoss()

def gen_img_plot(model, epoch, test_input):
    prediction = np.squeeze(model(test_input).detach().cpu().numpy())
    fig = plt.figure(figsize=(4, 4))
    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.imshow((prediction[i] + 1)/2) # 确保prediction[i] + 1)/2输出的结果是在0-1之间
        plt.axis('off')
    plt.show()
    
test_input = torch.randn(16, 100, device=device)

 # 保存每个epoch所产生的loss值
D_loss = []
G_loss = []

# 训练循环
for epoch in range(20): #训练20个epoch
    d_epoch_loss = 0 # 初始损失值为0
    g_epoch_loss = 0
    # len(dataloader)返回批次数，len(dataset)返回样本数
    count = len(dataloader)
    # 对dataloader进行迭代
    for step, (img, _) in enumerate(dataloader): # enumerate加序号
        img = img.to(device) #将数据上传到设备
        size = img.size(0) # 获取每一个批次的大小
        random_noise = torch.randn(size, 100, device=device)  # 随机噪声的大小是size个
        
        d_optim.zero_grad() # 将判别器前面的梯度归0
        
        real_output = dis(img)      # 判别器输入真实的图片，real_output是对真实图片的预测结果 
        
        # 得到判别器在真实图像上的损失
        # 判别器对于真实的图片希望输出的全1的数组，将真实的输出与全1的数组进行比较
        d_real_loss = loss_fn(real_output, 
                              torch.ones_like(real_output))      
        d_real_loss.backward() # 求解梯度
        
        
        gen_img = gen(random_noise)    
        # 判别器输入生成的图片，fake_output是对生成图片的预测
        # 优化的目标是判别器，对于生成器的参数是不需要做优化的，需要进行梯度阶段，detach()会截断梯度，
        # 得到一个没有梯度的Tensor，这一点很关键
        fake_output = dis(gen_img.detach()) 
        # 得到判别器在生成图像上的损失
        d_fake_loss = loss_fn(fake_output, 
                              torch.zeros_like(fake_output))      
        d_fake_loss.backward() # 求解梯度
        
        d_loss = d_real_loss + d_fake_loss # 判别器总的损失等于两个损失之和
        d_optim.step() # 进行优化
        
        g_optim.zero_grad() # 将生成器的所有梯度归0
        fake_output = dis(gen_img) # 将生成器的图片放到判别器中，此时不做截断，因为要优化生成器
        # 生层器希望生成的图片被判定为真
        g_loss = loss_fn(fake_output, 
                         torch.ones_like(fake_output))      # 生成器的损失
        g_loss.backward() # 计算梯度
        g_optim.step() # 优化
        
        # 将损失累加到定义的数组中，这个过程不需要计算梯度
        with torch.no_grad():
            d_epoch_loss += d_loss
            g_epoch_loss += g_loss
      
    # 计算每个epoch的平均loss，仍然使用这个上下文关联器
    with torch.no_grad():
        # 计算平均的loss值
        d_epoch_loss /= count
        g_epoch_loss /= count
        # 将平均loss放入到loss数组中
        D_loss.append(d_epoch_loss.item())
        G_loss.append(g_epoch_loss.item())
        # 打印当前的epoch
        print('Epoch:', epoch)
        # 调用绘图函数
        gen_img_plot(gen, epoch, test_input)


### 2.2 变分自编码器（VAE）

![](https://cdn.jsdelivr.net/gh/ADAning/Image/MLDL/vae2.png)
<center>自编码器基本结构</center>

![](https://pic1.zhimg.com/70/v2-e29d8d59a84a17b9199c8dbd5ade3eee_1440w.avis?source=172ae18b&biz_tag=Post)
<center>变分自编码器基本结构</center>

In [ ]:
# VAE代码实现
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import datetime

if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        # return F.sigmoid(self.fc4(h3))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


strattime = datetime.datetime.now()
model = VAE()
if torch.cuda.is_available():
    # model.cuda()
    print('cuda is OK!')
    model = model.to('cuda')
else:
    print('cuda is NO!')

reconstruction_function = nn.MSELoss(size_average=False)
# reconstruction_function = nn.MSELoss(reduction=sum)


def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        img = (img.cuda() if torch.cuda.is_available() else img)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        # train_loss += loss.data[0]
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            endtime = datetime.datetime.now()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} time:{:.2f}s'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 
                100. * batch_idx / len(dataloader),
                loss.item() / len(img), 
                (endtime-strattime).seconds))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './vae.pth')


### 2.3 流模型（flow-based model）

![](https://img-blog.csdnimg.cn/img_convert/4ad3616fbfd228d7619b6ec7a9967438.png)
<center>三种基本生成模型结构的对比</center>

### 2.4 扩散模型（Diffusion model）